In [1]:
%load_ext autoreload
%autoreload 2
from app import db
import numpy as np

from app.models import Tournament as TournamentModel
from app.models import Bracket as BracketModel
from app.models import Match as MatchModel
from app.models import Round as RoundModel
from app.models import User as UserModel

from tournament import Tournament

import numpy as np

from sqlalchemy.sql.expression import func, select

from tournament import (
    BracketTypes,
    Tournament,
    Match
)

In [2]:
import uuid

In [3]:
str(uuid.uuid4())

'75623681-eaba-4150-b92b-9bf9c2f51b55'

In [4]:
db.session.close()

In [5]:
def purge(Model):
    [db.session.delete(row) for row in Model.query.all()]

In [6]:
# purge all tables first

for Model in [BracketModel, TournamentModel, MatchModel, RoundModel, UserModel]:
    purge(Model)
db.session.commit()

In [7]:
# add all users to users table
users = [
    UserModel(username='TPN', email='TPN@example.com'.lower()), 
    UserModel(username='AngryFalco', email='AngryFalco@example.com'.lower()), 
    UserModel(username='Sunrisebanana', email='Sunrisebanana@example.com'.lower()), 
    UserModel(username='Ptolemy', email='Ptolemy@example.com'.lower()), 
    UserModel(username='Vik', email='Vik@example.com'.lower()), 
    UserModel(username='Kevin', email='Kevin@example.com'.lower()), 
    UserModel(username='Spaceghost', email='Spaceghost@example.com'.lower()), 
    UserModel(username='Burnaby', email='Burnaby@example.com'.lower()), 
]
[db.session.add(u) for u in users]
db.session.commit()

In [8]:
# Fall Charity LAN 2018 Melee Singles users
N_COMPETITORS = 8
seeds = [i+1 for i in range(N_COMPETITORS)]
usernames = [
    'TPN',
    'AngryFalco',
    'Sunrisebanana',
    'Ptolemy',
    'Vik',
    'Kevin',
    'Spaceghost',
    'Burnaby',
]
np.random.shuffle(usernames)
np.random.shuffle(seeds)

In [9]:
tuple_list = [(usernames[i], seeds[i]) for i in range(N_COMPETITORS)]

In [10]:
btype = BracketTypes.DOUBLE_ELIMINATION
t = Tournament(tuple_list, btype)

In [11]:
tournament_name = 'Fall Charity LAN 2018 Melee Singles'
TO = t.bracket.entrants[0]

# initial post to db
t.post_to_db(tournament_name, TO,)

In [12]:
# post self references in matches separately
for r in t.bracket.rounds:
    for m in r.matches:
        m.post_self_refs()

In [13]:
import sqlite3
import pandas as pd

# Create the connection
cnx = sqlite3.connect(r'./app.db')

q = \
"""
    select * from user where
    username like '%TPN%'
"""
q = \
"""
    select * from user
"""
# create the dataframe from a query
df = pd.read_sql_query(q, cnx)
df

,id,username,email,password_hash,about_me,last_seen
0,1,TPN,tpn@example.com,None,None,2020-02-20 01:19:33.768692
1,2,AngryFalco,angryfalco@example.com,None,None,2020-02-20 01:19:33.769136
2,3,Sunrisebanana,sunrisebanana@example.com,None,None,2020-02-20 01:19:33.769243
3,4,Ptolemy,ptolemy@example.com,None,None,2020-02-20 01:19:33.769319
4,5,Vik,vik@example.com,None,None,2020-02-20 01:19:33.769391
5,6,Kevin,kevin@example.com,None,None,2020-02-20 01:19:33.769461
6,7,Spaceghost,spaceghost@example.com,None,None,2020-02-20 01:19:33.769535
7,8,Burnaby,burnaby@example.com,None,None,2020-02-20 01:19:33.769602


In [14]:
pd.read_sql_query('select * from tournament', cnx)

,id,n_entrants,name,organizer_id
0,1,8,Fall Charity LAN 2018 Melee Singles,5


In [15]:
pd.read_sql_query('select * from bracket', cnx)

,id,bracket_type,tournament_id
0,1,DOUBLE_ELIMINATION,1


In [16]:
pd.read_sql_query('select * from bracket_entrants', cnx)

,user_id,bracket_id
0,5,1
1,6,1
2,3,1
3,2,1
4,7,1
5,8,1
6,1,1
7,4,1


In [17]:
pd.read_sql_query('select * from round', cnx)

,id,number,winners,bracket_id
0,1,1,1,1
1,2,2,1,1
2,3,3,1,1
3,4,4,1,1
4,5,5,1,1
5,6,1,0,1
6,7,2,0,1
7,8,3,0,1
8,9,4,0,1


In [18]:
df = pd.read_sql_query('select * from match', cnx)
df
# df.loc[df['uuid'] == '94de92a2-13ac-4e9a-9bf2-78636100a7b1']

,id,uuid,user_1,user_1_score,user_2,user_2_score,winner,round_id,winner_advance_to,loser_advance_to
0,1,86beefba-5d07-4c24-9373-ab0c78c27f33,5.0,None,4.0,None,None,1,5.0,10.0
1,2,ba8789f6-1d98-4f9a-bddd-75a27c41f623,6.0,None,1.0,None,None,1,5.0,10.0
2,3,b728b019-481e-4d52-ae88-6b62f0e3338a,3.0,None,8.0,None,None,1,6.0,11.0
3,4,525d7e14-f21e-4bfd-816d-fb8b09ca71b4,2.0,None,7.0,None,None,1,6.0,11.0
4,5,02c696c9-a0b4-4d08-9436-1191c802fe5f,NaN,None,NaN,None,None,2,7.0,12.0
5,6,6ae9f279-0fb8-48cc-a5e5-be71cca417e0,NaN,None,NaN,None,None,2,7.0,13.0
6,7,7370b0a6-470b-4c71-8f33-d2cabb2a9490,NaN,None,NaN,None,None,3,8.0,15.0
7,8,0a98d4a3-ad60-4db1-ac44-b4f6072c2137,NaN,None,NaN,None,None,4,NaN,NaN
8,9,c59c261e-0ff8-42d1-9065-d43306be3a5a,NaN,None,NaN,None,None,5,NaN,NaN
9,10,d5b5ba39-e8e3-4252-bb97-0275ec8e4b8c,NaN,None,NaN,None,None,6,12.0,NaN
